In [21]:
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import sys

sys.path.append("../python")

import utils


In [22]:
max_words = 2400
glove_dim = 25
hash_dim = 5

In [23]:
words_embed = utils.load_glove_embedding(utils.glove_embedding_path(25))
hash_embed = utils.load_glove_embedding('../data/models/hashtags/hash_vectors.d'+str(hash_dim)+'.txt')

In [24]:
df = pd.read_hdf("../data/preproc/tweets.hd5")

tk = Tokenizer(num_words=max_words,
               filters="", # already applied
               lower=True,
               split=" ",oov_token="<unk>")
tk.fit_on_texts(df["preproc_text"]) 

In [25]:
reload(utils)
words_embed_mtx, words_unk = utils.fill_embedding_matrix(tk.word_index,words_embed,max_words,glove_dim)
hash_embed_mtx, hash_unk = utils.fill_embedding_matrix(tk.word_index,hash_embed,max_words,hash_dim)


In [29]:
unk = { word: tk.word_index[word] for word in set(words_unk).intersection(set(hash_unk)) }
print(unk)

stop_mtx = np.zeros( (max_words+2, 2) )

# stop_mtx[unk.pop("<trunc>"),-2] = 1.
stop_mtx[unk.pop("<stop>"),-1] = 1.

stop_mtx[list(unk.values()),-2] = 1.


{'kaggle': 1388, 'rtit': 2330, 'nodexl': 1138, 'h2o': 1581, 'spirosmargaris': 1401, 'verisk': 1778, 'insurtech': 1838, '#rvp': 1717, '#domains': 1376, 'jupyter': 2162, 'chatbot': 794, '#nvidia': 2260, 'yape': 2165, '#sensor': 2171, 'kdnuggets': 965, '#machineintelligence': 2096, '#cfo': 2135, '#meetup': 2307, 'convolutional': 671, 'bytedance': 2371, 'kubernetes': 2253, 'interpretability': 1229, 'daysofcode': 2067, 'daysofmlcode': 1263, 'neurips': 467, '#mqtt': 2276, 'autoencoders': 2080, 'mikequindazzi': 182, 'pytorch': 1392, '#neuralnetwork': 1855, 'pierrepinna': 2313, 'ipfconline1': 1343, 'whova': 661, 'scikit': 1878, '#india': 1695, 'automl': 1819, 'intengineering': 2275, 'datascience': 1792, 'granvilledsc': 2140, '<stop>': 3, 'classifier': 2315, 'classifiers': 2362, '#ravivisvesvarayasharadaprasad': 1718, '#fraud': 2196, 'analyticbridge': 2226, 'machinelearning': 509, 'neurips2018': 1754, 'kirkdborne': 635, 'v1': 1311, 'cnewsuae': 2329, 'variational': 1627, 'datasciencectrl': 1218,

In [30]:
unk_idx = list(unk.values())
unk

{'#aiethics': 2187,
 '#article': 2227,
 '#ceo': 2078,
 '#cfo': 2135,
 '#cios': 2394,
 '#domains': 1376,
 '#fraud': 2196,
 '#india': 1695,
 '#jupyter': 2273,
 '#ksql': 2170,
 '#language': 2367,
 '#li': 1624,
 '#machineintelligence': 2096,
 '#manufacturing': 2185,
 '#meetup': 2307,
 '#mqtt': 2276,
 '#neuralnetwork': 1855,
 '#nvidia': 2260,
 '#ravivisvesvarayasharadaprasad': 1718,
 '#rvp': 1717,
 '#sensor': 2171,
 '#techiegeeks': 1960,
 'alphazero': 2161,
 'analyticbridge': 2226,
 'autoencoders': 2080,
 'automl': 1819,
 'bengio': 1853,
 'bytedance': 2371,
 'chatbot': 794,
 'chatbots': 705,
 'classifier': 2315,
 'classifiers': 2362,
 'cnewsuae': 2329,
 'convolutional': 671,
 'datascience': 1792,
 'datasciencectrl': 1218,
 'datax': 1724,
 'daysofcode': 2067,
 'daysofmlcode': 1263,
 'deeplearn007': 1649,
 'dimensionality': 2219,
 'ethereum': 2139,
 'fisher85m': 1069,
 'granvilledsc': 2140,
 'h2o': 1581,
 'insurtech': 1838,
 'intengineering': 2275,
 'interpretability': 1229,
 'ipfconline1': 1

In [31]:
np.isnan(words_embed_mtx).sum()

0

In [32]:
np.isnan(hash_embed_mtx).sum()

0

In [33]:
# ! mkdir ../data/models/sequences

import pickle
import json

with open("../data/models/sequences/info.json","w+") as out:
    save = dict(max_words=max_words,unk_idx=unk_idx)
    out.write(json.dumps(save))
    
with open('../data/models/sequences/tokenizer.pkl','wb+') as out:
    out.write( pickle.dumps(tk) )
    
np.save('../data/models/sequences/embed_mtx.npy',words_embed_mtx)
np.save('../data/models/sequences/hash_mtx.npy',hash_embed_mtx)
np.save('../data/models/sequences/hash_mtx.npy',stop_mtx)

    


In [34]:
from sklearn.model_selection import train_test_split

df["sequences"] = tk.texts_to_sequences(df["preproc_text"])

df.to_hdf("../data/preproc/sequences.hd5","squences",columns=["sequences"],mode="w")

# index = np.arange(df.index.shape[0]).astype(np.int)
# train_idx, test_idx = train_test_split(index,test_size=0.2,random_state=123456)
# df_train = df.iloc[train_idx]
# df_test = df.iloc[test_idx]
# df_train.to_hdf("../data/preproc/sequences.hd5","train",columns=["sequences"])
# df_test.to_hdf("../data/preproc/sequences.hd5","test",columns=["sequences"])


/home/musella/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['lang', 'text', 'user/name', 'preproc_text', 'sequences']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [35]:
list( zip(df.iloc[:20]["text"],tk.sequences_to_texts( tk.texts_to_sequences(df.iloc[:20]["preproc_text"]) )) )

[('#neurips2018 attendees, a new discussion on "HCI and monitization in A.I." was just started. Join the conversation… https://t.co/zftGzImYQ0',
  '<hashtag> #neurips2018 attendees a new discussion on <unk> and <unk> in a i was just started join the <url> <stop>'),
 ('Interesting stats about my #NeurIPS2018 tweets! People mostly tend to check the NeurIPS tweets exactly after the co… https://t.co/PQk31bTpDb',
  'interesting stats about my <hashtag> #neurips2018 tweets people mostly <unk> to check the neurips tweets exactly after the <url> <stop>'),
 ('Jordan Gruber @j2g2_ : Our robot "Marvin" can count and identify asbestos particles - this can cut costs to $15 - v… https://t.co/0o2iWrPkYL',
  '<unk> <unk> <user> our robot <unk> can <unk> and identify <unk> <unk> this can cut costs to <number> <url> <stop>'),
 ("RT @hypergiant: What are Hypergiant's big ambitions for an intelligent future all about? Check out @MyABJ discussing machine intelligence a…",
  'rt <user> what are <unk> s big 

In [36]:
# ?tk.texts_to_sequences

In [37]:
tk.texts_to_sequences( ["<unk>"] )

[[1]]

In [39]:
# tk.word_index